In [3]:
import os
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# ================================
# 1. Load data
# ================================

kpis_path = "../data/processed/compustat_kpis.csv"
macro_path = "../data/processed/macro_data.csv"

df = pd.read_csv(kpis_path)
df_macro = pd.read_csv(macro_path)

print("Compustat KPIs shape:", df.shape)
print("Macro data shape     :", df_macro.shape)

df = df.replace([np.inf, -np.inf], np.nan)

# ================================
# 2. Filter on dlrsn and basic typing
# ================================

df["dlrsn"] = pd.to_numeric(df["dlrsn"], errors="coerce")
df["gvkey"] = df["gvkey"].astype(str)
df["fyear"] = pd.to_numeric(df["fyear"], errors="coerce").astype("Int64")

# On garde seulement les firmes dont dlrsn ∈ {NaN, 2, 3} sur toute l'histoire
def firm_dlrsn_valid(s: pd.Series) -> bool:
    return s.isna().all() or s.dropna().isin([2, 3]).all()

firm_valid = df.groupby("gvkey")["dlrsn"].apply(firm_dlrsn_valid)
valid_firms = firm_valid[firm_valid].index

df = df[df["gvkey"].isin(valid_firms)].copy()
print("\nAfter firm-level dlrsn filter (NaN/2/3 only):", df.shape)

# ================================
# 3. Keep firms with no missing KPI
# ================================

kpi_cols = [
    "roa",
    "total_debt_to_equity",
    "current_ratio",
    "cfo_margin",
    "asset_turnover",
]
kpi_cols = [c for c in kpi_cols if c in df.columns]

df[kpi_cols] = df[kpi_cols].replace([np.inf, -np.inf], np.nan)

firm_has_missing = df.groupby("gvkey")[kpi_cols].apply(
    lambda g: g.isna().any().any()
)
firms_no_missing = firm_has_missing[~firm_has_missing].index

print("\nFirms before KPI-missing filter:", df["gvkey"].nunique())
print("Firms with full KPIs           :", len(firms_no_missing))

df = df[df["gvkey"].isin(firms_no_missing)].copy()
df = df.sort_values(["gvkey", "fyear"]).reset_index(drop=True)

print("Shape after removing firms with missing KPIs:", df.shape)

# ================================
# 4. Identify bankrupt vs healthy firms
#    - bankrupt: all dlrsn ∈ {2,3}
#    - healthy : all dlrsn NaN
# ================================

g = df.groupby("gvkey")["dlrsn"]

is_bankrupt = g.apply(lambda s: s.notna().all() and s.isin([2, 3]).all())
is_healthy = g.apply(lambda s: s.isna().all())

bankrupt_firms = is_bankrupt[is_bankrupt].index.values
healthy_firms = is_healthy[is_healthy].index.values

print("\nFirm types BEFORE balancing:")
print("  Bankrupt (all years 2/3):", len(bankrupt_firms))
print("  Healthy  (all years NaN):", len(healthy_firms))

if len(bankrupt_firms) == 0 or len(healthy_firms) == 0:
    raise ValueError("Not enough bankrupt or healthy firms to balance the sample.")

# ================================
# 5. Balance sample (same nb of firms)
# ================================

np.random.seed(42)
n_sample = min(len(bankrupt_firms), len(healthy_firms))

sampled_bankrupt = np.random.choice(bankrupt_firms, size=n_sample, replace=False)
sampled_healthy = np.random.choice(healthy_firms, size=n_sample, replace=False)

selected_firms = np.concatenate([sampled_bankrupt, sampled_healthy])

df_bal = df[df["gvkey"].isin(selected_firms)].copy()
df_bal = df_bal.sort_values(["gvkey", "fyear"]).reset_index(drop=True)

print("\nFirm types AFTER balancing:")
print("  Bankrupt sampled:", len(sampled_bankrupt))
print("  Healthy sampled :", len(sampled_healthy))
print("Balanced firm-level shape:", df_bal.shape)

# ================================
# 6. KPI deltas by firm
# ================================

delta_kpi_cols = []
for col in kpi_cols:
    dcol = f"delta_{col}"
    df_bal[dcol] = df_bal.groupby("gvkey")[col].diff()
    delta_kpi_cols.append(dcol)

print("\nDelta KPI columns created:", delta_kpi_cols)

before_rows = df_bal.shape[0]
df_bal = df_bal.dropna(subset=delta_kpi_cols).copy()
after_rows = df_bal.shape[0]

print("Shape after dropping first-year rows (no KPI deltas):")
print("  Before:", before_rows)
print("  After :", after_rows)

# ================================
# 7. Prepare macro data + deltas
# ================================

df_macro.columns = (
    df_macro.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

if "name" not in df_macro.columns:
    raise ValueError("Macro data must contain a 'Name' column with years (now 'name').")

df_macro["fyear"] = pd.to_numeric(df_macro["name"], errors="coerce")
df_macro = df_macro.dropna(subset=["fyear"]).copy()
df_macro["fyear"] = df_macro["fyear"].astype(int)
df_macro = df_macro.drop(columns=["name"])

macro_cols = [c for c in df_macro.columns if c != "fyear"]

for col in macro_cols:
    df_macro[col] = (
        df_macro[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .str.replace(" ", "", regex=False)
    )
    df_macro[col] = pd.to_numeric(df_macro[col], errors="coerce")

for col in macro_cols:
    df_macro[col] = df_macro[col].fillna(df_macro[col].median())

df_macro = df_macro.sort_values("fyear").reset_index(drop=True)

delta_macro_cols = []
for col in macro_cols:
    dcol = f"delta_{col}"
    df_macro[dcol] = df_macro[col].diff()
    delta_macro_cols.append(dcol)

print("\nDelta macro columns created:", delta_macro_cols)

before_rows_macro = df_macro.shape[0]
df_macro = df_macro.dropna(subset=delta_macro_cols).copy()
after_rows_macro = df_macro.shape[0]

print("Macro shape after dropping first year (no deltas):")
print("  Before:", before_rows_macro)
print("  After :", after_rows_macro)

# ================================
# 8. Merge firm + macro on fyear
# ================================

df_bal["fyear"] = df_bal["fyear"].astype(int)
df_panel = df_bal.merge(df_macro, on="fyear", how="left")

print("\nPanel shape after merge firm × macro:", df_panel.shape)

# ================================
# 9. Create target y (1 = bankrupt)
# ================================

bankrupt_set = set(sampled_bankrupt)
df_panel["y"] = df_panel["gvkey"].isin(bankrupt_set).astype(int)

print("\nTarget distribution (y):")
print(df_panel["y"].value_counts())

# ================================
# 10. Save final panel
# ================================

output_path = "../data/final/panel_balanced_with_deltas.csv"
os.makedirs("../data/final/", exist_ok=True)
df_panel.to_csv(output_path, index=False)

print("\nFinal panel saved to:", output_path)
print("Final shape:", df_panel.shape)
print("\nPreview:")
print(df_panel.head())

# =========================================
# 11. Create file containing ONLY delta KPIs + delta MACRO
# =========================================

# Sélectionne toutes les colonnes delta
delta_cols_only = [c for c in df_panel.columns if c.startswith("delta_")]

df_deltas = df_panel[["gvkey", "fyear"] + delta_cols_only].copy()

# Chemin de sortie
output_deltas_path = "../data/final/panel_only_deltas.csv"
os.makedirs("../data/final/", exist_ok=True)

df_deltas.to_csv(output_deltas_path, index=False)

print("\nDelta-only file saved to:", output_deltas_path)
print("Shape:", df_deltas.shape)
print(df_deltas.head())


Compustat KPIs shape: (332675, 24)
Macro data shape     : (31, 6)

After firm-level dlrsn filter (NaN/2/3 only): (183200, 24)

Firms before KPI-missing filter: 13953
Firms with full KPIs           : 3279
Shape after removing firms with missing KPIs: (52025, 24)

Firm types BEFORE balancing:
  Bankrupt (all years 2/3): 283
  Healthy  (all years NaN): 2996

Firm types AFTER balancing:
  Bankrupt sampled: 283
  Healthy sampled : 283
Balanced firm-level shape: (7406, 24)

Delta KPI columns created: ['delta_roa', 'delta_total_debt_to_equity', 'delta_current_ratio', 'delta_cfo_margin', 'delta_asset_turnover']
Shape after dropping first-year rows (no KPI deltas):
  Before: 7406
  After : 6840

Delta macro columns created: ['delta_us_gdp_(ar)_cura', 'delta_us_cpi___all_urban:_all_items_sadj', 'delta_us_treasury_bill_rate___3_month_(ep)_nadj', 'delta_us_treasury_yield_adjusted_to_constant_maturity___20_year_nadj', 'delta_us_unemployment_rate_sadj']
Macro shape after dropping first year (no delt